# Bosonic Mixture with Excitation Transfer

In [1]:
using Pkg; Pkg.activate()
using KadanoffBaym, FFTW, Interpolations
using LinearAlgebra

using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")
using LaTeXStrings

  Activating project at `~/.julia/environments/v1.10`


### Hamiltonian

\begin{align}
    \hat{H} &=  \omega_0 \sum_{i=1}^{L} \left(\hat{b}^\dagger_{i} \hat{b}^{\phantom{\dagger}}_{i} - \hat{a}^\dagger_{i} \hat{a}^{\phantom{\dagger}}_{i} \right) + J \sum_{\langle i, j \rangle} \hat{a}^\dagger_{i}\hat{b}^{\phantom{\dagger}}_{i}\hat{b}^{\dagger}_{j}\hat{a}^{\phantom{\dagger}}_{j}
\end{align}

### Green functions

\begin{align}
    \mathcal{A}_{i,\,j}(t, t') &=  -i \delta_{ij}\left\langle{\mathcal{T}_\mathcal{C}\hat{a}_i^{}(t)\hat{a}_i^\dagger(t')}\right\rangle \\
    \mathcal{B}_{i,\,j}(t, t') &= -i \delta_{ij}\left\langle{\mathcal{T}_\mathcal{C}\hat{b}_i^{}(t)\hat{b}_i^\dagger(t')}\right\rangle
\end{align}    

### Self-energies

\begin{align}
    \left[{\boldsymbol{\Sigma}^\lessgtr_{\mathcal{A}}(t, t')}\right]_{i,\, i} &= - J^2\sum_{j \in {\mathcal{N}}_i}\mathcal{B}^\lessgtr_{i,\, i}(t, t') \mathcal{A}^\lessgtr_{j,\, j}(t, t')\mathcal{B}^\gtrless_{j,\, j}(t', t), \\
    \left[{\boldsymbol{\Sigma}^\lessgtr_{\mathcal{B}}(t, t')}\right]_{i,\, i} &= - J^2\sum_{j \in {\mathcal{N}}_i}\mathcal{A}_{i,\, i}^\lessgtr(t, t') \mathcal{A}^\gtrless_{j,\, j}(t', t)\mathcal{B}^\lessgtr_{j,\, j}(t, t')
\end{align}  

In [2]:
# Container for problem data
struct ProblemData
    GL::GreenFunction{ComplexF64, 4, Array{ComplexF64, 4}, SkewHermitian}
    GG::GreenFunction{ComplexF64, 4, Array{ComplexF64, 4}, SkewHermitian}
    ΣL::GreenFunction{ComplexF64, 4, Array{ComplexF64, 4}, SkewHermitian}
    ΣG::GreenFunction{ComplexF64, 4, Array{ComplexF64, 4}, SkewHermitian}
    L::Int64
    H::Matrix{ComplexF64}
    J::Float64
  
    # Initialize problem
    function ProblemData(GL0::Matrix{ComplexF64}, L::Int64, H::Matrix{ComplexF64}, J::Float64)
        @assert H == H' "Non-hermitian Hamiltonian"

        data = new(
          GreenFunction(reshape(GL0, size(GL0)..., 1, 1), SkewHermitian),
          GreenFunction(reshape(GL0 - 1.0im * I, size(GL0)..., 1, 1), SkewHermitian),
          GreenFunction(zeros(ComplexF64, size(GL0)..., 1, 1), SkewHermitian),
          GreenFunction(zeros(ComplexF64, size(GL0)..., 1, 1), SkewHermitian),
          L,
          H,
          J
        )

        # Initialize self-energies
        self_energies!(data, 1.0zeros(1), 1.0zeros(1), 1.0zeros(1), 1, 1)

        return data
    end
end

# Self-energy
function self_energies!(data, _, _, _, t1, t2)
    (; GL, GG, ΣL, ΣG, L, H, J) = data

    # adjust array size
    if (n = size(GL, 3)) > size(ΣL, 3)
        resize!(ΣL, n)
        resize!(ΣG, n)
    end
  
    # index mapping
    NNs = [1, 0] # nearest neighbours
    N_mu = mu -> NNs[mu % L + 1] + mu - mu % L
    idxs = [[(mu + L) % 2L, N_mu(mu), (N_mu(mu) + L) % 2L] .+ 1 for mu in 0:2L-1]
    idxs = hcat(idxs...)
    
    # self-energies using the index mapping
    ΣL[t1, t2] = -J^2 * diag(GL[t1, t2])[idxs[1, :]] .* diag(GL[t1, t2])[idxs[2, :]] .* diag(GG[t2, t1])[idxs[3, :]] |> diagm
    ΣG[t1, t2] = -J^2 * diag(GG[t1, t2])[idxs[1, :]] .* diag(GG[t1, t2])[idxs[2, :]] .* diag(GL[t2, t1])[idxs[3, :]] |> diagm
end

self_energies! (generic function with 1 method)

In [3]:
# integration boundaries
tspan = (0.0, 5.0)

# problem data
data = begin

    # parameters
    L = 2
    h = 5.0
    H = ComplexF64[-h 0 0 0; 0 -h 0 0; 0 0 h 0; 0 0 0 h];
    J = 1.

    # initial condition
    delta_n = 0.1
    GL0 = -1.0im .* 2e-1 .* [2 0 0 0; 0 1 0 0; 0 0 2 0; 0 0 0 0.5]

    ProblemData(GL0, L, H, J)
end;

In [4]:
# right-hand side for the "vertical" evolution
function fv!(out, data, ts, h1, h2, t1, t2)
  (; GL, GG, ΣL, ΣG, L, H) = data
  
  # real-time collision integral
  ∫dt1(A,B,C) = sum(h1[s] * ((A[t1, s] - B[t1, s]) * C[s, t2]) for s in 1:t1)
  ∫dt2(A,B,C) = sum(h2[s] * (A[t1, s] * (B[s, t2] - C[s, t2])) for s in 1:t2)

  out[1] = -1.0im * (H * GL[t1, t2] + ∫dt1(ΣG, ΣL, GL) + ∫dt2(ΣL, GL, GG))
  out[2] = -1.0im * (H * GG[t1, t2] + ∫dt1(ΣG, ΣL, GG) + ∫dt2(ΣG, GL, GG))
end

# right-hand side for the "diagonal" evolution
function fd!(out, data, ts, h1, h2, t1, t2)
  fv!(out, data, ts, h1, h2, t1, t2)
  out .-= adjoint.(out)
end

# call the solver
sol = kbsolve!(
  (out, x...) -> fv!(out, data, x...), 
  (out, x...) -> fd!(out, data, x...), 
  [data.GL, data.GG], 
  tspan; 
  callback = (x...) -> self_energies!(data, x...), 
  atol=1e-10, 
  rtol=1e-8,
  stop = x -> (println("t: $(x[end])"); flush(stdout); false)
);

t: 0.0
t: 1.0e-6
t: 5.999999999999999e-6
t: 1.9481442224736467e-5
t: 3.713479735107213e-5
t: 0.00012540157298275047
t: 0.00020484167105126099
t: 0.0006020421613938136
t: 0.002588044613106576
t: 0.004375446819648063
t: 0.013312457852355495
t: 0.05799751301589265
t: 0.05799751301589265
t: 0.05799751301589265
t: 0.08548806248047995
t: 0.11111851793569844
t: 0.136594325586376
t: 0.1608944390653646
t: 0.18276454119645436
t: 0.21229157580551333
t: 0.2505775493102775
t: 0.2850349254645652
t: 0.31891495504512984
t: 0.3528844986905081
t: 0.3878078498661392
t: 0.4257227866561987
t: 0.4653913009441241
t: 0.5075831036521246
t: 0.5521143701384993
t: 0.5921925099762365
t: 0.6345286987096268
t: 0.6801741437018803
t: 0.7242891101493056
t: 0.7669090061594699
t: 0.8076619516416014
t: 0.8485341103579622
t: 0.8910778120323197
t: 0.9337851159351439
t: 0.9780907084463684
t: 1.022305445360943
t: 1.0669264901420514
t: 1.1123199996903466
t: 1.1567999240257814
t: 1.2033552324428425
t: 1.2484907462326529
t: 1.29

## QuTiP benchmark

In [5]:
using PyCall
qt = pyimport("qutip")

LoadError: PyError (PyImport_ImportModule

The Python package qutip could not be imported by pyimport. Usually this means
that you did not install qutip in the Python version being used by PyCall.

PyCall is currently configured to use the Julia-specific Python distribution
installed by the Conda.jl package.  To install the qutip module, you can
use `pyimport_conda("qutip", PKG)`, where PKG is the Anaconda
package that contains the module qutip, or alternatively you can use the
Conda package directly (via `using Conda` followed by `Conda.add` etcetera).

Alternatively, if you want to use a different Python distribution on your
system, such as a system-wide Python (as opposed to the Julia-specific Python),
you can re-configure PyCall with that Python.   As explained in the PyCall
documentation, set ENV["PYTHON"] to the path/name of the python executable
you want to use, run Pkg.build("PyCall"), and re-launch Julia.

) <class 'ModuleNotFoundError'>
ModuleNotFoundError("No module named 'q

In [ ]:
# time parameters
times = range(first(sol.t), stop=last(sol.t), length=128+1)
n = length(times) - 1

### Hamiltonian

In [ ]:
n_max = 2; # Fock-space truncation

# initial state
psi0 = qt.tensor([(sqrt(1 - 1.0im * data.GL[1, 1][k, k]) * qt.basis(n_max + 1, 0) 
            + sqrt(1.0im * data.GL[1, 1][k, k]) * qt.basis(n_max + 1, 1)).unit() for k in 1:2*L]);

# operators
ids = [qt.qeye(n_max + 1), qt.qeye(n_max + 1), qt.qeye(n_max + 1), qt.qeye(n_max + 1)]
ops = [deepcopy(ids) for _ in 1:4]

for (i, op) in enumerate(ops)
    op[i] = qt.destroy(n_max + 1)
end

ops = qt.tensor.(ops)
b1, b2, a1, a2 = ops;

In [ ]:
# make diagonal density matrix (i.e. dropping coherences)
rho0 = psi0 * psi0.dag();
for j in 1:rho0.shape[2] , i in 1:rho0.shape[1]
    i != j ? rho0.data[i, j] = 0.0 : continue
end

In [ ]:
# Hamiltonian (hard-coded for two sites)
H  = -h * b1.dag() * b1
H += -h * b2.dag() * b2
H += +h * a1.dag() * a1
H += +h * a2.dag() * a2
H += J * (a1.dag() * b1 * b2.dag() * a2 + b1.dag() * a1 * a2.dag() * b2)

# observables
obs = [b1.dag() * b1, b2.dag() * b2, a1.dag() * a1, a2.dag() * a2];

### Simulation

In [ ]:
# quickly solve once for observables
me = qt.mesolve(H, rho0, times, [], obs)

# solve for the time-dependent density matrix
t_sols = qt.mesolve(H, rho0, times); # t_sols.states returns density matrices

#### Two times

In [ ]:
# compute two-time average -i<X(t2)Y(t1)>
function two_time_average(X, Y, rho_t1)
    X_t2_Y_t1 = zeros(ComplexF64, n + 1, n + 1)
    for k in 1:(n + 1)
        Y_rho_t1 = qt.mesolve(H, Y * rho_t1.states[k], times).states
        for l in 1:(n + 1)
            X_t2_Y_t1[k, l] = -1.0im * (X * Y_rho_t1[l]).tr()    
        end
    end
    
    # transform to regular two-time square used by the KB solver
    rotated_X_t2_Y_t1 = zeros(ComplexF64, n + 1, 2*(n + 1) - 1)
    for (k, x) in enumerate([X_t2_Y_t1[k, :] for k in 1:(n + 1)])
        for (l, y) in enumerate(x)
            ind = k + l - 1
            rotated_X_t2_Y_t1[k, ind] = y 
        end
    end    
    
    return rotated_X_t2_Y_t1[:, 1:n+1]
end;

In [ ]:
create_destroy = [zeros(ComplexF64, n + 1, n + 1) for _ in 1:1]#2L]
destroy_create = [zeros(ComplexF64, n + 1, n + 1) for _ in 1:1]#2L];

In [ ]:
destroyers = [b1, b2, a1, a2]
for k in 1:1#2L
    create_destroy[k] = two_time_average(destroyers[k].dag(), destroyers[k], t_sols)
    destroy_create[k] = two_time_average(destroyers[k], destroyers[k].dag(), t_sols)
    
    # flip real part when creation operator is to the right (equivalent to flipping the sign of tau)
    destroy_create[k] = -conj(destroy_create[k])
end

In [ ]:
b1_dag_b1 = create_destroy[1]
b1_b1_dag = destroy_create[1];

In [ ]:
# compute missing two-time triangle from symmetry
full_square = A -> (A - adjoint(A) - (A |> diag |> diagm));

In [ ]:
figure(figsize=(6, 2))
subplot(121)
imshow(real(full_square(b1_dag_b1)), cmap="plasma")

subplot(122)
imshow(real(full_square(b1_b1_dag)), cmap="plasma")

tight_layout()

## Plotting

In [ ]:
# final time
T = sol.t[end]

# scale parameters
t_scale = (J == 0. ? 1. : J)
ω_scale = (J == 0. ? 1. : 1/J);

### Equal time

In [ ]:
xpad = 8
ypad = 5

figure(figsize=(7, 3))

ax = subplot(121)
idx = 1
ax.plot(sol.t, data.GL.data[idx, idx ,:,:] |> ((-) ∘ imag ∘ diag), ls="-", label="\$i="*string((idx - 1) % 2 + 1)*"\$", c="C0")
ax.plot(times, me.expect[idx], "--", c="C0", lw=3, alpha=0.5)
idx = 2
ax.plot(sol.t, data.GL.data[idx, idx ,:,:] |> ((-) ∘ imag ∘ diag), ls=":", label="\$i="*string((idx - 1) % 2 + 1)*"\$", c="C1")
ax.plot(times, me.expect[idx], "-.", c="C1", lw=3, alpha=0.5)
ax.set_xlabel("\$Jt\$") 
ylabel("\$-\\mathrm{Im}\\; \\mathcal{A}^<_{i,\\, i}(t, t)\$", labelpad=10)
ax.set_xticks([0, 2.5, 5])
ax.set_xlim(0, sol.t[end]) 
ax.set_ylim(0, 0.5)
ax.legend(loc="best", handlelength=1.4, frameon=false, borderpad=0, labelspacing=0.25)

ax = subplot(122)
idx = 3
ax.plot(sol.t, data.GL.data[idx, idx ,:,:] |> ((-) ∘ imag ∘ diag), ls="-", label="\$i="*string((idx - 1) % 2 + 1)*"\$", c="C2")
ax.plot(times, me.expect[idx], "--", c="C2", lw=3, alpha=0.5)
idx = 4
ax.plot(sol.t, data.GL.data[idx, idx ,:,:] |> ((-) ∘ imag ∘ diag), ls=":", label="\$i="*string((idx - 1) % 2 + 1)*"\$", c="C3")
ax.plot(times, me.expect[idx], "-.", c="C3", lw=3, alpha=0.5)
ax.set_xlabel("\$Jt\$") 
ylabel("\$-\\mathrm{Im}\\; \\mathcal{B}^<_{i,\\, i}(t, t)\$", labelpad=16)
ax.yaxis.set_label_position("right")
ax.set_xticks([0, 2.5, 5])
ax.set_yticklabels([])
ax.set_xlim(0, sol.t[end]) 
ax.set_ylim(0, 0.5)
ax.legend(loc="best", handlelength=1.4, frameon=false, borderpad=0, labelspacing=0.25)
tight_layout(pad=0.25, w_pad=1, h_pad=0)

# savefig("interacting_bosons_example_1.pdf")

### Relative time

In [ ]:
# quantum number to look at
idx = 1;

In [ ]:
ρ_11_kb = interpolate((sol.t, sol.t), view(data.GL.data .- data.GG.data, idx, idx, :, : ), Gridded(Linear()));
ρ_11_qt = interpolate((times, times), view(full_square(create_destroy[idx] .- destroy_create[idx]), :, : ), Gridded(Linear()));

In [ ]:
new_times = range(first(sol.t), stop=last(sol.t), length=2048);

ρ_11_kb_wigner, (taus, ts) = wigner_transform([ρ_11_kb(t1, t2) for t1 in new_times, t2 in new_times]; ts=new_times, fourier=false);
ρ_11_qt_wigner, (taus_qt, ts_qt) = wigner_transform([ρ_11_qt(t1, t2) for t1 in new_times, t2 in new_times]; ts=new_times, fourier=false);

In [ ]:
xpad = 8
ypad = 5

center = floor(length(new_times) / 2) |> Int

figure(figsize=(7, 3))

ax = subplot(121)
plot(t_scale * taus, ρ_11_kb_wigner[:, center] |> imag, ls="-", c="C0", lw=1.5)
plot(t_scale * taus_qt, ρ_11_qt_wigner[:, center] |> imag, ls="--", c="C0", lw=2.5, alpha=0.5)

ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_xlabel(L"J \tau")
ax.set_ylabel(L"\mathrm{Re}\,A_{\mathcal{A}_{1,\, 1}}(T, \tau)_W")
ax.set_xlim(-t_scale * T, t_scale * T)
ax.set_ylim(-1.0, 1.0)

ax = subplot(122)
plot(t_scale * taus, -ρ_11_kb_wigner[:, center] |> real, ls="-", c="C0", lw=1.5)
plot(t_scale * taus_qt, -ρ_11_qt_wigner[:, center] |> real, ls="--", c="C0", lw=2.5, alpha=0.5)
ax.set_xlabel(L"J \tau")
ax.set_xlim(-t_scale * T, t_scale * T)
ax.set_ylim(-1.0, 1.0)
ax.set_yticklabels([])
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_ylabel(L"\mathrm{Im}\,A_{\mathcal{A}_{1,\, 1}}(T, \tau)_W", labelpad=16)
ax.yaxis.set_label_position("right")

tight_layout(pad=0.1, w_pad=1, h_pad=0)

# savefig("interacting_bosons_example_2.pdf")